# On-Graph Numerical Scheme

The purpose of this notebook is to attempt to execute the numerical scheme for the non-local graph problem, incorporating the method of `DtNEvalsViaMinimiser.ipynb`.
This only executes for our "Cross in the plane" geometry, as established in the aforementioned notebook, and we study the following problem.

Take a quasimomentum value $\theta$, and (for the time being) let $\omega>0$ be fixed.
For each bulk region $\Omega_i$ where $\Omega = \bigcup_{i\in\Lambda}\Omega_i$, let $\lambda_n^i, \varphi_n^i$ be the $n^{th}$ eigenpair of the Dirichlet-to-Neumann (DtN) map $\mathcal{D}_{\omega}^i$ for the Helmholtz operator $\Delta_{\theta} + \omega^2$ in $\Omega_i$.
Then we wish to find $u\in H^2(\mathbb{G})$ that solves

\begin{align}
    -\left( \dfrac{\mathrm{d}}{\mathrm{d}y} + \mathrm{i}\theta_{jk} \right)^2 u^{(jk)}
    &= \omega^2 u^{(jk)} - \left( \mathcal{D}_{\omega}^+ + \mathcal{D}_{\omega}^- \right)u,
    &\qquad \text{on each } I_{jk}, \\
    \sum_{j\sim k} \left( \dfrac{\partial}{\partial n} + \mathrm{i}\theta_{jk} \right)u^{(jk)}(v_j) &= 0,
    &\qquad \text{at each } v_j,
\end{align}

(plus the standard continuity of the function $u$ at the vertices).

## Spectral/FE Method Approach

We will choose a finite dimensional subspace $V\subset H^2(\mathbb{G})$ that consists of "tent" or "hat" functions.
Place nodes $\tau_j$ at each $v_j\in\mathcal{V}$.
Then place further nodes $\tau_{n}^{jk}\in I_{jk}$ for $n=1,...,N_{jk}\in\mathbb{N}$, where $\tau_{n-1}^{jk} < \tau_{n}^{jk}$ and $\tau_{n}^{jk}\neq v_0$.

Define the ("hat") functions

\begin{align*}
    \phi_n^{jk}(y) = 
    \begin{cases}
        \frac{ y - \tau_{n-1}^{jk} }{ \tau_{n}^{jk} - \tau_{n-1}^{jk} } & y\in\left[ \tau_{n-1}^{jk} , \tau_{n}^{jk} \right], \\
        \frac{ \tau_{n+1}^{jk} - y }{ \tau_{n+1}^{jk} - \tau_{n}^{jk} } & y\in\left[ \tau_{n}^{jk} , \tau_{n+1}^{jk} \right], \\
        0 & \text{otherwise},       
    \end{cases}
    &\qquad\implies\qquad
    (\phi_n^{jk})'(y) = 
    \begin{cases}
        \frac{ 1 }{ \tau_{n}^{jk} - \tau_{n-1}^{jk} } & y\in\left[ \tau_{n-1}^{jk} , \tau_{n}^{jk} \right], \\
        \frac{ -1 }{ \tau_{n+1}^{jk} - \tau_{n}^{jk} } & y\in\left[ \tau_{n}^{jk} , \tau_{n+1}^{jk} \right], \\
        0 & \text{otherwise},    
    \end{cases}
\end{align*}

for each $I_{jk}\in\mathcal{E}$ and $n=1,...,N_{jk}$.
At the vertices $v_j$ we place the special vertex hat functions;

\begin{align*}
    \phi^j(y) &=
    \begin{cases}
        \frac{ y - \tau_1^{jk} }{ v_j - \tau_1^{jk} } & y\in\left[ v_j , \tau_1^{jk} \right], \ j\rightarrow k, \\
        \frac{ y - \tau_{N_{jk}}^{jk} }{ v_j - \tau_{N_{jk}}^{jk} } & y\in\left[ \tau_{N_{jk}}^{jk} , v_j \right], \ j\leftarrow k, \\
        0 & \text{otherwise}.
    \end{cases}
\end{align*}

__NOTE:__ It may also be worth considering Fourier functions, seeing as we will be storing the DtN eigenfunctions as Fourier series.

We then let 

\begin{align*}
V = \mathrm{span}\left\{ \phi^j, \phi_n^{jk} \ \middle\vert \ v_j\in\mathcal{V}, \ I_{jk}\in\mathcal{E}, \ n=1,...,N_{jk} \right\}, 
&\qquad
M^* := \mathrm{dim} V = \lvert\mathcal{V}\rvert + \sum_{j\rightarrow k} N_{jk},
\end{align*}

which provides us with our finite-dimensional approximating space.
Write our solution in $V$, $u_V$, as 

\begin{align*}
u_V = \sum_{m=1}^{M^*} u_m \psi_m
\end{align*}

We can then compute various inner products of these "basis functions":

\begin{align*}
    \langle \phi_n^{jk}, \phi_{n-1}^{jk} \rangle_{L^2(\mathbb{G})} 
    &= \frac{1}{6} \left( \tau_n^{jk} - \tau_{n-1}^{jk} \right), &\qquad 2\leq n\leq N_{jk}, \\
    \langle \phi_n^{jk}, \phi_{n}^{jk} \rangle_{L^2(\mathbb{G})} 
    &= \frac{1}{3} \left( \tau_{n+1}^{jk} - \tau_{n-1}^{jk} \right), &\qquad 1\leq n\leq N_{jk}, \\
    \langle \phi_1^{jk}, \phi^j \rangle_{L^2(\mathbb{G})} 
    &= \frac{1}{6} \left( v_j - \tau_1^{jk} \right), \\
    \langle \phi_{N_{jk}}^{jk}, \phi^k \rangle_{L^2(\mathbb{G})} 
    &= \frac{1}{6} \left( v_k - \tau_{N_{jk}}^{jk} \right), \\
    \langle \phi^j, \phi^j \rangle_{L^2(\mathbb{G})} 
    &= \frac{1}{3} \sum_{j\rightarrow k} \left( \tau_1^{jk} - v_j \right) + \frac{1}{3} \sum_{j\leftarrow k} \left( v_j - \tau_{N_{jk}}^{jk} \right),
\end{align*}

and any other inner products are identically zero.
We also need to compute the following

\begin{align*}
    \langle \phi_n^{jk}, (\phi_{n-1}^{jk})' \rangle_{L^2(\mathbb{G})}
    &= -\frac{1}{2}, \\
    \langle (\phi_n^{jk})', \phi_{n-1}^{jk} \rangle_{L^2(\mathbb{G})}
    &= \frac{1}{2}, \\
    \langle (\phi_n^{jk})', (\phi_{n-1}^{jk})' \rangle_{L^2(\mathbb{G})}
    &= -\frac{1}{\tau_n^{jk} - \tau_{n-1}^{jk}},
\end{align*}

so that we can observe that

\begin{align*}
    \langle \nabla^{\theta}\phi_n^{jk}, \nabla^{\theta}\phi_{n-1}^{jk} \rangle_{L^2(\mathbb{G})}
    &= \theta_{jk}^2 \langle \phi_n^{jk}, \phi_{n-1}^{jk} \rangle_{L^2(\mathbb{G})}
    + \mathrm{i}\theta_{jk} \langle \phi_n^{jk}, (\phi_{n-1}^{jk})' \rangle_{L^2(\mathbb{G})}
    - \mathrm{i}\theta_{jk} \langle (\phi_n^{jk})', \phi_{n-1}^{jk} \rangle_{L^2(\mathbb{G})}
    + \theta_{jk}^2 \langle \phi_n^{jk}, \phi_{n-1}^{jk} \rangle_{L^2(\mathbb{G})} \\
    &= -\frac{1}{\tau_n^{jk} - \tau_{n-1}^{jk}}
    - \frac{\mathrm{i}\theta_{jk}}{2}
    - \frac{\mathrm{i}\theta_{jk}}{2}
    + \frac{\theta_{jk}^2}{6} \left( \tau_n^{jk} - \tau_{n-1}^{jk} \right) \\
    &= \frac{\theta_{jk}^2}{6} \left( \tau_n^{jk} - \tau_{n-1}^{jk} \right) - \mathrm{i}\theta_{jk} - \frac{1}{\tau_n^{jk} - \tau_{n-1}^{jk}}
\end{align*}

Let $\{\psi_m\}_{m=1}^{M^*}$ be an appropriate indexing for the functions $\phi^j, \phi_{n}^{jk}$.

We then define the constants

\begin{align*}
    c_{m,n}^i &= \langle \psi_m, \varphi_n^i \rangle_{L^2\left(\partial\Omega_i\right)},
    &\qquad \psi_m = \sum_n c_{m,n}^i\varphi_n^i, \\
    \tilde{c}_{n,m}^i &= \langle \varphi_n^i, \psi_m \rangle_{V},
    &\qquad \varphi_n^i = \sum_m \tilde{c}_{n,m}^i \psi_m,
\end{align*}

which allows us to easily compute the action of the DtN maps, provided we have the eigenfunctions to hand.
Additionally, due to the support of the functions $\varphi_n^i$ when extended by zero to $V$, we can also notice that $c_{m,n}^i = \overline{\tilde{c}_{n,m}^i}$.
However, we need to truncate this basis to ever have a hope of proceeding computationally, so choose a truncation index $N_i\in\mathbb{N}$ for each $\Omega_i$.
It is then a case of us assembling the matrices

\begin{align*}
    B_{n,m} &= \langle \nabla^{\theta}\psi_m, \nabla^{\theta}\psi_n \rangle_{L^2(\mathbb{G})}, \\
    C_{n,m} &= \langle \psi_m, \psi_n \rangle_{L^2(\mathbb{G})}, \\
    L_{n,m} &= \sum_{v_j\in\mathcal{V}} \sum_{j\rightarrow k}
    \left\{ \sum_{\hat{n}=1}^{N_+}c_{m,\hat{n}}^+ \lambda_{\hat{n}}^+ \sum_{l=1}^{M^*}\tilde{c}_{\hat{n},l}^+ \langle \psi_l, \psi_n \rangle_{L^2(I_{jk})} + \sum_{\hat{n}=1}^{N_-}c_{m,\hat{n}}^- \lambda_{\hat{n}}^- \sum_{l=1}^{M^*}\tilde{c}_{\hat{n},l}^- \langle \psi_l, \psi_n \rangle_{L^2(I_{jk})}  \right\}, \\
    M_{n,m} &= B_{n,m} - \omega^2 C_{n,m} + L_{n,m},
\end{align*}

and solving the system $M(\omega) U = 0$, where $U = \left( u_1,...u_{M^*} \right)$ provides us with our approximate solution.

__NOTE:__ The matrices $B$ and $C$ are constant with respect to $\omega$, it is only the matrix $L$ that changes with $\omega$, as one needs to recompute the DtN eigenfunctions and eigenvalues.
Of course, $C$ is also premultiplied by $\omega^2$ to complicate things, so we will have to solve the above system as a generalised eigenvalue problem for matrix-valued $M(\omega)$.

### Storing the DtN Eigenfunctions

The DtN eigenfunctions $\varphi_n^i$ are approximated by a truncated Fourier basis and stored in a class `FourierFunction`, which we should import.
Documentation on this class can be found in the notebook `DtNEvalsViaMinimiser.ipynb`, however since we cannot import a class from another notebook, we need to import it from the `.py` file that we created to hold the functions and methods used in computing the DtN eigenfunctions.

At any rate, each $\varphi_n^i$ is represented by

\begin{align*}
    \varphi_n^i &= \sum_{\alpha=-M_i}^{M_i}\sum_{\beta=-M_i}^{M_i} c_{\alpha\beta}\mathrm{e}^{2\pi\mathrm{i}(\alpha x+ \beta y)} =: \sum_{\alpha=-M_i}^{M_i}\sum_{\beta=-M_i}^{M_i} c_{\alpha\beta}\eta_{\alpha\beta}
\end{align*}

where the `FourierFunction` class stores both the matrix of constants $c_{\alpha\beta}$ and the appropriately reshaped column vector.
The value $M_i$ can be read from the instance of the class itself, although in our example we only have a single bulk region, so we just use $M_i=M$.
The class _also_ stores the eigenvalue $\lambda_n^i$ (or approximation thereof) in the attribute `.lbda`.

This representation does mean that we can analytically evaluate most of the inner products we will need, which should save us some computation time in the long run.

## Computing Inner Products

We will need to compute the inner products (on the boundary/graph) between the $\psi_m$ and functions $\eta_{\alpha\beta}$.
This is further complicated because we need to consider the boundaries seperately, and then sum to get the relevant norms.

To begin, let us consider the $x=0$ boundary. 
We find that (for $\phi_n^{jk}$ supported on the $x=0$ edge),

\begin{align*}
    \langle \phi_n^{(jk)}, \eta_{\alpha\beta} \rangle &=
    \frac{ (\tau_{n-1}^{jk}-\tau_{n}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\beta \tau_{n+1}^{jk}} + (\tau_{n}^{jk}-\tau_{n+1}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\beta \tau_{n-1}^{jk}} + (\tau_{n+1}^{jk}-\tau_{n-1}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\beta \tau_{n}^{jk}} }{ 2\pi\mathrm{i}\beta(\tau_{n}^{jk}-\tau_{n-1}^{jk})(\tau_{n+1}^{jk}-\tau_{n}^{jk}) },
\end{align*}

with the inner product being zero otherwise.
Analogously, on the $x=1$ boundary we find that

\begin{align*}
    \langle \phi_n^{(jk)}, \eta_{\alpha\beta} \rangle &=
    \frac{ (\tau_{n-1}^{jk}-\tau_{n}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\beta \tau_{n+1}^{jk}} + (\tau_{n}^{jk}-\tau_{n+1}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\beta \tau_{n-1}^{jk}} + (\tau_{n+1}^{jk}-\tau_{n-1}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\beta \tau_{n}^{jk}} }{ 2\pi\mathrm{i}\beta(\tau_{n}^{jk}-\tau_{n-1}^{jk})(\tau_{n+1}^{jk}-\tau_{n}^{jk}) } \mathrm{e}^{-2\pi\mathrm{i}\alpha}.
\end{align*}

Then by symmetry, we also know that for the $y=0$ edge we have that

\begin{align*}
    \langle \phi_n^{(jk)}, \eta_{\alpha\beta} \rangle &=
    \frac{ (\tau_{n-1}^{jk}-\tau_{n}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\alpha \tau_{n+1}^{jk}} + (\tau_{n}^{jk}-\tau_{n+1}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\alpha \tau_{n-1}^{jk}} + (\tau_{n+1}^{jk}-\tau_{n-1}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\alpha \tau_{n}^{jk}} }{ 2\pi\mathrm{i}\alpha(\tau_{n}^{jk}-\tau_{n-1}^{jk})(\tau_{n+1}^{jk}-\tau_{n}^{jk}) },
\end{align*}

and for the $y=1$ boundary,

\begin{align*}
    \langle \phi_n^{(jk)}, \eta_{\alpha\beta} \rangle &=
    \frac{ (\tau_{n-1}^{jk}-\tau_{n}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\alpha \tau_{n+1}^{jk}} + (\tau_{n}^{jk}-\tau_{n+1}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\alpha \tau_{n-1}^{jk}} + (\tau_{n+1}^{jk}-\tau_{n-1}^{jk})\mathrm{e}^{-2\pi\mathrm{i}\alpha \tau_{n}^{jk}} }{ 2\pi\mathrm{i}\alpha(\tau_{n}^{jk}-\tau_{n-1}^{jk})(\tau_{n+1}^{jk}-\tau_{n}^{jk}) } \mathrm{e}^{-2\pi\mathrm{i}\beta}.
\end{align*}

# Proposition: Fourier Basis for $L^2(\mathbb{G})$.

Let $\mathbb{G} = \left(\mathcal{V}, \mathcal{E}\right)\subset\Omega=\left[0,1\right)^2$ be the (finite) period graph of a periodic graph in $\mathbb{R}^2$.
For each $I_{jk}\in\mathcal{E}$ let $l_{jk}=\lvert I_{jk}\rvert$ and define the functions

\begin{align*}
    \psi_{\alpha}^{jk}(y) &=
    \begin{cases}
        0 & y\not\in I_{jk}, \\
        \mathrm{e}^{\frac{2\pi\mathrm{i}\alpha y}{l_{jk}}} & y\in I_{jk}.
    \end{cases}
\end{align*}

The the family of functions $\mathcal{P} := \left\{ \psi_{\alpha}^{jk} \right\}_{\alpha\in\mathbb{Z}}^{I_{jk}\in\mathcal{E}} \subset L^2\left(\mathbb{G}\right)$ is an orthonormal basis of $L^2\left(\mathbb{G}\right)$.

###### Proof

We can first compute the inner product in $L^2\left(\mathbb{G}\right)$ between two members $\psi_{\alpha}^{jk},\psi_{\beta}^{mn}\in\mathcal{P}$ as

\begin{align*}
    \langle \psi_{\alpha}^{jk},\psi_{\beta}^{mn} \rangle_{L^2\left(\mathbb{G}\right)}
    &= \sum_{p\rightarrow q} \int_{I_{pq}} \psi_{\alpha}^{jk}\overline{\psi}_{\beta}^{mn} \ \mathrm{d} y
    = \delta_{jm}\delta_{kn} \int_{I_{jk}} \psi_{\alpha}^{jk}\overline{\psi}_{\beta}^{jk} \ \mathrm{d} y \\
    &= \delta_{jm}\delta_{kn} \int_0^{l_{jk}} \mathrm{e}^{\frac{2\pi\mathrm{i}\alpha y}{l_{jk}}}\mathrm{e}^{-\frac{2\pi\mathrm{i}\beta y}{l_{jk}}} \ \mathrm{d} y
    = \delta_{jm}\delta_{kn}\delta_{\alpha\beta},
\end{align*}

from which it is clear that $\mathcal{P}$ is an orthonormal family.
It thus remains to show that $\mathcal{P}$ is spanning, to this end let us suppose that there exists a $v\in L^2\left(\mathbb{G}\right)$ such that

\begin{align*}
    \langle v, \psi_{\alpha}^{jk} \rangle_{L^2\left(\mathbb{G}\right)} = 0 \qquad\forall \psi_{\alpha}^{jk}\in\mathcal{P}.
\end{align*}

Let us fix an edge $I_{jk}$ for the moment, and notice that

\begin{align*}
    0 &= \langle v, \psi_{\alpha}^{jk} \rangle_{L^2\left(\mathbb{G}\right)}
    = \langle v^{(jk)}, \psi_{\alpha}^{jk} \rangle_{L^2\left(\mathbb{G}\right)}
    = \langle v^{(jk)}, \psi_{\alpha}^{jk} \rangle_{L^2\left(\left[0,l_{jk}\right]\right)},
    \qquad \forall\alpha\in\mathbb{Z}.
\end{align*}

Furthermore, the collection $\left\{ \psi_\alpha^{jk} \right\}_{\alpha\in\mathbb{Z}}$ is a basis of $L^2\left(\left[0,l_{jk}\right]\right)$, and $v^{(jk)}\in L^2\left(\left[0,l_{jk}\right]\right)$ by definition of $v$.
As such, we conclude that $v^{(jk)}=0$.
Since the edge $I_{jk}\in\mathcal{E}$ was arbitrary, we therefore conclude that $v^{(jk)}=0$ for every edge $I_{jk}$, and thus $v=0\in L^2\left(\mathbb{G}\right)$.

Thus, $\mathcal{P}$ is also a spanning set of $L^2\left(\mathbb{G}\right)$.
Given the orthonormality we demonstrated earlier, $\mathcal{P}$ therefore forms an orthonormal basis of $L^2\left(\mathbb{G}\right)$.